In [1]:
import numpy as np
from pyscf import gto
from pyscf import dft

mol = gto.Mole()
mol.atom = """
H     -0.70000   0.000000   0.000000
H     0.700000   0.000000   0.000000
C     0.000000   1.000000   0.000000
"""
mol.basis = "6-31G"
mol.spin = 0
mol.build()

#
# First converge a high-spin UKS calculation
#
mks = dft.RKS(mol)
mks.xc = "lda"
mks.kernel()

g = mks.nuc_grad_method()
g.kernel()

converged SCF energy = -38.1963346277403
--------------- RKS gradients ---------------
         x                y                z
0 H     0.0303926482    -0.0241273906     0.0000000000
1 H    -0.0303926482    -0.0241273906     0.0000000000
2 C     0.0000000000     0.0482568276     0.0000000000
----------------------------------------------


array([[ 3.03926482e-02, -2.41273906e-02,  1.53101442e-16],
       [-3.03926482e-02, -2.41273906e-02,  1.79660216e-16],
       [ 4.29835332e-16,  4.82568276e-02,  4.85109178e-16]])

In [2]:
fock_real = mks.get_fock()

eri_ao = mks.mol.intor("int2e")
dm1 = mks.make_rdm1()
fock_check = (
    mks.mol.intor("int1e_kin")
    + mks.mol.intor("int1e_nuc")
    + np.einsum("uvkl, kl -> uv", eri_ao, dm1)
)
fock_check = np.array([fock_check, fock_check])

In [15]:
grids = dft.Grids(mol)
grids.kernel()
ao_0 = dft.numint.eval_ao(mks.mol, grids.coords, deriv=0)
dm1_ks = mks.make_rdm1()

scf_r = dft.numint.eval_rho(mks.mol, ao_0, dm1_ks)
# oe_fock = oe.contract_expression(
#     "p,p,pa,pb->ab",
#     np.shape(dft2cc.ao_0[:, 0]),
#     np.shape(dft2cc.ao_0[:, 0]),
#     dft2cc.ao_0,
#     dft2cc.ao_0,
#     constants=[2, 3],
#     optimize="optimal",
# )
exc_lda = dft.libxc.eval_xc("lda", scf_r)[0] * grids.weights
exc_lda_cal = - 3 / 4 * (3 / np.pi) ** (1 / 3) * scf_r ** (1 / 3) * grids.weights

In [16]:
np.sum(exc_lda_cal), np.sum(exc_lda)

(np.float64(-68.73995864890199), np.float64(-68.73331122857232))